In [1]:
import pandas as pd
import time 
import warnings
from tqdm.notebook import trange, tqdm
from os import listdir
from os.path import isfile, join
import glob

warnings.filterwarnings('ignore')

In [2]:
vocab = {
    0 : 'Пн',
    1 : 'Вт',
    2 : 'Ср',
    3 : 'Чт',
    4 : 'Пт',
    5 : 'Сб',
    6 : 'Вс'
}


In [3]:
def create_target_set(target_name, how_to_shift, hours = 48):
    Target_KS = pd.read_csv(target_name+"P_in_out.csv",sep = ";")
    Target_KS["DateTime"] = pd.to_datetime(Target_KS["DateTime"])   
    Target_KS["dayofweek"] = Target_KS["DateTime"].dt.dayofweek
    Target_KS["Pin"] = Target_KS["Pin"].str.replace(",",".").astype('float32')
    Target_KS["Pout"] = Target_KS["Pout"].str.replace(",",".").astype('float32')
    abs = pd.DataFrame()
    abs["DateTime"] = pd.date_range(Target_KS["DateTime"].min(), periods=((Target_KS["DateTime"].max() - Target_KS["DateTime"].min()).seconds/3600 + (Target_KS["DateTime"].max() - Target_KS["DateTime"].min()).days*24 +1), freq="h")
    Target_KS = pd.merge(abs,Target_KS,how='left',on="DateTime")
    Target_KS = Target_KS.ffill() #Тут заполнение пропусков чтобы дискретность была час     
    if how_to_shift == "1":
        Target_KS.loc[Target_KS['dayofweek'] == 0,'DateTime'] -= pd.DateOffset(hours=72)   
        Target_KS.loc[Target_KS['dayofweek'] == 1,'DateTime'] -= pd.DateOffset(hours=96) 
        Target_KS.loc[Target_KS['dayofweek'] == 2,'DateTime'] -= pd.DateOffset(hours=48)
        Target_KS.loc[Target_KS['dayofweek'] == 3,'DateTime'] -= pd.DateOffset(hours=48)
        Target_KS.loc[Target_KS['dayofweek'] == 4,'DateTime'] -= pd.DateOffset(hours=48)
        Target_KS.loc[Target_KS['dayofweek'] == 5,'DateTime'] -= pd.DateOffset(hours=48)
        Target_KS.loc[Target_KS['dayofweek'] == 6,'DateTime'] -= pd.DateOffset(hours=48)
        Target_KS["dayofweek"] = Target_KS["DateTime"].dt.dayofweek
        Target_KS["dayofweek"] = Target_KS["dayofweek"].replace(vocab, regex=True)
        Target_KS.columns = [f'{c}_'+ target_name if c!="DateTime" else c for c in Target_KS]
        Target_KS = Target_KS.drop(Target_KS[(Target_KS["DateTime"].dt.year < 2021)].index)
        return Target_KS
    elif how_to_shift == "2":
        Target_KS['DateTime'] -= pd.DateOffset(hours=hours)
        Target_KS["dayofweek"] = Target_KS["DateTime"].dt.dayofweek
        Target_KS["dayofweek"] = Target_KS["dayofweek"].replace(vocab, regex=True)
        Target_KS.rename(columns={"Pin": "Pin_lag_"+str(hours)+"h", "Pout": "Pout_lag_"+str(hours)+"h"}, inplace = True)
        Target_KS.columns = [f'{c}_'+ target_name if c!="DateTime" else c for c in Target_KS]
        Target_KS = Target_KS.drop(Target_KS[(Target_KS["DateTime"].dt.year < 2021)].index)
        return Target_KS
    else:        
        Target_KS["dayofweek"] = Target_KS["dayofweek"].replace(vocab, regex=True)
        Target_KS.columns = [f'{c}_'+ target_name if c!="DateTime" else c for c in Target_KS]
        return Target_KS
   
    

In [4]:
def prepare_dataset(target_name, how_to_shift_target,hours = 48):
    grs_list = glob.glob("ГРС/*")
    auto_plan_list = glob.glob("Автоплан/*")
    target_set = create_target_set(target_name, how_to_shift_target,hours)
    schema_info = pd.read_csv(target_name+".csv",sep = ";")
    schema_info["DateTime"] = pd.to_datetime(schema_info["DateTime"])
    target_set = pd.merge(target_set,schema_info,how='left',on="DateTime")
    for grs in tqdm(grs_list):
        feature = pd.read_csv(grs, sep = ";")
        feature["DateTime"] = pd.to_datetime(feature["DateTime"])
        feature.columns = [f'{c}_'+ grs.split("\\")[1][:-4] if c!="DateTime" else c for c in feature]
        target_set = pd.merge(target_set,feature,how='left',on="DateTime")
    for auto_list in tqdm(auto_plan_list):
        feature = pd.read_csv(auto_list, sep = ";")
        if "Date" in feature.columns:
            feature["Data"] = feature["Date"]
            feature.drop(columns=["Date"],inplace=True)
        try:
            feature["DateTime"] = pd.to_datetime(feature["Data"])
        except KeyError:
            continue
        feature.drop(columns=["Data"],inplace=True)
        feature.columns = [f'{c}_'+ auto_list.split("\\")[1][:-4] if c!="DateTime" else c for c in feature]
        target_set = pd.merge(target_set,feature,how='left',on="DateTime")
    # target_set = target_set.ffill()
    return target_set
        
    

In [5]:
t_set = prepare_dataset("КС-15", "2")

  0%|          | 0/119 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [6]:
t_set

,DateTime,Object_КС-15,Pin_lag_48h_КС-15,Pout_lag_48h_КС-15,dayofweek_КС-15,Object,Tnv,GPAinwork,PNA,Schema1,...,82_КС-19 Факт по фидерам,83_КС-19 Факт по фидерам,84_КС-19 Факт по фидерам,85_КС-19 Факт по фидерам,86_КС-19 Факт по фидерам,87_КС-19 Факт по фидерам,88_КС-19 Факт по фидерам,89_КС-19 Факт по фидерам,ID_Кто планировал,Login_Кто планировал
0,2021-01-01 00:00:00,2010.0,36.500000,46.400002,Пт,2010.0,-9.0,2.0,-6,21.0,...,0,0,0,0,0,0,0,0,14.0,Немытых
1,2021-01-01 01:00:00,2010.0,36.500000,46.400002,Пт,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,NaN,NaN
2,2021-01-01 02:00:00,2010.0,36.500000,46.700001,Пт,2010.0,-9.0,2.0,-6,21.0,...,0,0,0,0,0,0,0,0,NaN,NaN
3,2021-01-01 03:00:00,2010.0,36.500000,46.700001,Пт,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,NaN,NaN
4,2021-01-01 04:00:00,2010.0,36.299999,46.799999,Пт,2010.0,-9.0,2.0,-6,21.0,...,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27150,2024-02-03 10:00:00,2010.0,46.900002,46.900002,Сб,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27151,2024-02-03 11:00:00,2010.0,46.900002,46.900002,Сб,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27152,2024-02-03 12:00:00,2010.0,46.700001,46.700001,Сб,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27153,2024-02-03 13:00:00,2010.0,46.700001,46.700001,Сб,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
t_set.to_excel("КС-15_set_v1.xlsx",index=None)

In [8]:
weather_dict = {
    "akjar":        "Погода/Акъяр.xlsx",
    "borovoe":      "Погода/Боровое.xlsx",
    "bredy":        "Погода/Бреды.xlsx",
    "krasnoepole":  "Погода/Красноеполе.xlsx",
    "orsk":         "Погода/Орск.xlsx",
    "pogoda":       "Погода/Погода p2.xlsx",
    "chelyabinsk":  "Погода/Челябинск.xlsx",
    "youjnostep":   "Погода/Южно-Степное.xlsx",
    "magnitogorsk": "Погода/7Магнитогорскойптицефабрики(Первомайский).xlsx",
}

In [9]:
akjar =         pd.read_excel("Погода/Акъяр.xlsx")
borovoe =       pd.read_excel("Погода/Боровое.xlsx")
bredy =         pd.read_excel("Погода/Бреды.xlsx")
krasnoepole =   pd.read_excel("Погода/Красноеполе.xlsx")
orsk =          pd.read_excel("Погода/Орск.xlsx")
pogoda =        pd.read_excel("Погода/Погода p2.xlsx")
chelyabinsk =   pd.read_excel("Погода/Челябинск.xlsx")
youjnostep =    pd.read_excel("Погода/Южно-Степное.xlsx")
magnitogorsk =  pd.read_excel("Погода/7Магнитогорскойптицефабрики(Первомайский).xlsx")

In [10]:
weather_dfs = {}
for key, value in weather_dict.items():
    active_df = pd.ExcelFile(value)
    print(active_df.sheet_names)
    for i in active_df.sheet_names:
        if len(active_df.sheet_names) < 2:
            weather_dfs.update({key: {i: pd.read_excel(value,  sheet_name=i)}})
        else:
            for j in active_df.sheet_names:
                weather_dfs.update({f"{key}_{j}": {j: pd.read_excel(value,  sheet_name=j)}})

['Лист1']
['Лист1']
['Лист1']
['Лист1']
['Лист1']
['Молжив', 'Солнечный', 'Троицк', 'темерязевское', 'долгодеревеское', 'Медведевский']
['Лист1']
['Лист1']
['Лист1']


In [11]:
weather_dfs.keys()

dict_keys(['akjar', 'borovoe', 'bredy', 'krasnoepole', 'orsk', 'pogoda_Молжив', 'pogoda_Солнечный', 'pogoda_Троицк', 'pogoda_темерязевское', 'pogoda_долгодеревеское', 'pogoda_Медведевский', 'chelyabinsk', 'youjnostep', 'magnitogorsk'])

In [49]:
import datetime

In [50]:
type(weather_dfs["akjar"]["Лист1"].loc[996]['Средняя\nтемпература']) == type(datetime.datetime(2024, 2, 13, 0, 0))


True

In [13]:
data = weather_dfs["akjar"]["Лист1"].loc[996]['Средняя\nтемпература']

In [14]:
print(str(data.day)+"."+str(data.month))

13.2


In [53]:
def correct_data_into_value(date):
    if type(date) == type(datetime.datetime(2024, 2, 13, 0, 0)):
        return str(f"{date.day}.{date.month%12}")
    return date

In [34]:
test_df = weather_dfs

In [54]:
for i in weather_dfs.keys():
    for j in weather_dfs[i].keys():
        print(f"----{i}----")
        first_column = 1
        for k in weather_dfs[i][j]:
            if first_column == 1:
                first_column = 0
                continue
            test_df[i][j][k] = weather_dfs[i][j][k].apply(lambda x: correct_data_into_value(x))


----akjar----
----borovoe----
----bredy----
----krasnoepole----
----orsk----
----pogoda_Молжив----
----pogoda_Солнечный----
----pogoda_Троицк----
----pogoda_темерязевское----
----pogoda_долгодеревеское----
----pogoda_Медведевский----
----chelyabinsk----
----youjnostep----
----magnitogorsk----


In [66]:
weather_dfs

{'akjar': {'Лист1':           Дата Максимальная\nтемпература Минимальная\nтемпература  \
  0   2021-01-01                     -11.3                    -13.6   
  1   2021-01-02                     -11.7                    -18.9   
  2   2021-01-03                     -11.8                    -20.6   
  3   2021-01-04                     -13.0                    -22.4   
  4   2021-01-05                     -13.2                    -23.0   
  ..         ...                       ...                      ...   
  995 2023-09-26                      25.0                      8.0   
  996 2023-09-27                      19.0                      7.0   
  997 2023-09-28                      19.0                      6.0   
  998 2023-09-29                      17.0                      6.0   
  999 2023-09-30                      17.0                      6.0   
  
      Средняя\nтемпература  Атмосферное\nдавление  Скорость\nветра  Осадки  \
  0                  -12.7                  997.1

In [69]:
test_df["akjar"]["Лист1"].tail(10)

,Дата,Максимальная\nтемпература,Минимальная\nтемпература,Средняя\nтемпература,Атмосферное\nдавление,Скорость\nветра,Осадки,Эффективная\nтемпература
990,2023-09-21,19.0,4.0,12.1,NaN,2.0,0.0,10.2
991,2023-09-22,23.0,8.0,17.1,NaN,5.0,0.0,13.6
992,2023-09-23,22.0,11.0,16.6,NaN,3.0,0.0,15.2
993,2023-09-24,23.0,10.0,17.4,NaN,2.0,0.0,16.7
994,2023-09-25,24.0,14.0,18.4,NaN,2.0,0.0,17.6
995,2023-09-26,25.0,8.0,17.0,NaN,3.0,0.0,15.1
996,2023-09-27,19.0,7.0,13.2,NaN,2.0,0.0,11.8
997,2023-09-28,19.0,6.0,11.6,NaN,3.0,0.0,9.1
998,2023-09-29,17.0,6.0,11.6,NaN,3.0,0.0,9.1
999,2023-09-30,17.0,6.0,11.6,NaN,3.0,0.0,9.4


In [64]:
t_set[]

,DateTime,Object_КС-15,Pin_lag_48h_КС-15,Pout_lag_48h_КС-15,dayofweek_КС-15,Object,Tnv,GPAinwork,PNA,Schema1,...,82_КС-19 Факт по фидерам,83_КС-19 Факт по фидерам,84_КС-19 Факт по фидерам,85_КС-19 Факт по фидерам,86_КС-19 Факт по фидерам,87_КС-19 Факт по фидерам,88_КС-19 Факт по фидерам,89_КС-19 Факт по фидерам,ID_Кто планировал,Login_Кто планировал
0,2021-01-01 00:00:00,2010.0,36.500000,46.400002,Пт,2010.0,-9.0,2.0,-6,21.0,...,0,0,0,0,0,0,0,0,14.0,Немытых
1,2021-01-01 01:00:00,2010.0,36.500000,46.400002,Пт,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,NaN,NaN
2,2021-01-01 02:00:00,2010.0,36.500000,46.700001,Пт,2010.0,-9.0,2.0,-6,21.0,...,0,0,0,0,0,0,0,0,NaN,NaN
3,2021-01-01 03:00:00,2010.0,36.500000,46.700001,Пт,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,NaN,NaN
4,2021-01-01 04:00:00,2010.0,36.299999,46.799999,Пт,2010.0,-9.0,2.0,-6,21.0,...,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27150,2024-02-03 10:00:00,2010.0,46.900002,46.900002,Сб,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27151,2024-02-03 11:00:00,2010.0,46.900002,46.900002,Сб,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27152,2024-02-03 12:00:00,2010.0,46.700001,46.700001,Сб,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27153,2024-02-03 13:00:00,2010.0,46.700001,46.700001,Сб,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
for i in test_df.keys():
    for j in test_df[i].keys():
        merge=pd.merge(t_set, test_df[i][j], on="Дата")
        print(merge)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [16]:
type(weather_dfs["akjar"]['Лист1'].loc[999]["Средняя\nтемпература"])

datetime.datetime

In [17]:
# df.rolling(3, min_periods = 1, center = false)